In [ ]:
import numpy as np
import pandas as pd
import time
import pickle

from collections import Counter

In [ ]:
train = pd.read_csv("h-and-m-personalized-fashion-recommendations/transactions_train.csv")

In [ ]:
train['customer_id'] = train['customer_id'].str[-16:].apply(lambda x: int(x, 16)).astype('int64')
train['article_id'] = train.article_id.astype('int32')
train.t_dat = pd.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id']]

In [ ]:
train.head()

In [ ]:
agg_articles = sub_train.groupby('customer_id')['article_id'].apply(list).reset_index(name='agg_articles')

In [ ]:
cnt = Counter()

for _, row in agg_articles.iterrows():    
    articles = row['agg_articles']
    for i in range(0, len(articles) - 1):
        for j in range(i+1, len(articles)):
            a1, a2 = articles[i], articles[j]
            if a1 not in cnt:
                cnt[a1] = Counter()
            if a2 not in cnt:
                cnt[a2] = Counter()
            
            cnt[a1][a2] += 1
            cnt[a2][a1] += 1

In [ ]:
most_common_purchase = {}

for a1, articles in cnt.items():
    li = [(i, j) for (i, j) in articles.items()]
    li = sorted(li, key=lambda x: x[1], reverse=True)
    
    li = li[: min(len(li), 7)]
    
    most_common_purchase[a1] = li
    
most_common_purchase

In the below cells, we compute the frequent item that is commonly purchased together

In [ ]:
agg_total = train.groupby('customer_id')['article_id'].apply(list).reset_index(name='agg_articles')

In [ ]:
cnt = Counter()
start = time.time()

for idx, row in agg_total.iterrows():
    articles = row['agg_articles']
    for i in range(0, len(articles) - 1):
        for j in range(i+1, len(articles)):
            a1, a2 = articles[i], articles[j]
            if a1 not in cnt:
                cnt[a1] = Counter()
            if a2 not in cnt:
                cnt[a2] = Counter()
            
            cnt[a1][a2] += 1
            cnt[a2][a1] += 1
    
    if idx % 10000 == 0:
        end = time.time()
        print(f"Row {idx} processed")
        print(f"Time passed {end - start}")
        print(f"Current size of the cnt {len(cnt)}")
        start = time.time()

In [ ]:
import pickle

with open('most_common_purchase.pkl', 'wb') as file:
    pickle.dump(most_common_purchase_new, file)

In [ ]:
with open('most_common_purchase.pkl', 'rb') as file:
    res = pickle.load(file)

In [ ]:
most_common_purchase = {}

for a1, articles in cnt.items():
    li = [(i, j) for (i, j) in articles.items()]
    li = sorted(li, key=lambda x: x[1], reverse=True)
    
    li = li[: min(len(li), 3)]
    
    most_common_purchase[a1] = li

In the below cells, we compute the last purchased item from last week

In [ ]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id', 'max_dat']
train = train.merge(tmp, on=['customer_id'], how='left')
# train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
# sub_train = sub_train.loc[sub_train['diff_dat'] <= 6]

In [ ]:
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days

In [ ]:
last_week_purchase = train.loc[train['diff_dat'] <= 6]

In [ ]:
last_week_purchase

In [ ]:
last_week_purchase_agg = last_week_purchase.groupby('customer_id')['article_id'].apply(list).reset_index(name='agg_articles')

In [ ]:
last_week_purchase_count = last_week_purchase.groupby('article_id').size().reset_index(name='count')

In [ ]:
last_week_top = last_week_purchase_count.sort_values(by=['count'], ascending=False)[:20]['article_id'].to_list()

In [ ]:
customers = pd.read_csv('h-and-m-personalized-fashion-recommendations/customers.csv')
customers = customers['customer_id'].str[-16:].apply(lambda x: int(x, 16)).astype('int64')
customers = customers.to_list()

In [114]:
predictions = []

for idx, customer in enumerate(customers):
    ans = []
    p = last_week_purchase_agg[last_week_purchase_agg['customer_id'] == customer]
    if p.size:
        articles = p['agg_articles'].to_list()[0]
        tmp = []
        for article in articles:
            tmp.extend(most_common_purchase.get(article, []))
        tmp = sorted(tmp, key=lambda x: x[1], reverse=True)
        tmp = tmp[:min(len(tmp), 12)]
        ans.extend([elem[0] for elem in tmp])
    
    if len(ans) < 12:
        diff = 12 - len(ans)
        for item in most_popular_all_time:
            if item not in ans:
                ans.append(item)
                diff -= 1
                if diff <= 0:
                    break
    
    predictions.append(ans)
    
    if idx % 10000 == 0:
        print(f"Processed {idx} customers")
            

Processed 0 customers
Processed 10000 customers
Processed 20000 customers
Processed 30000 customers
Processed 40000 customers
Processed 50000 customers
Processed 60000 customers
Processed 70000 customers
Processed 80000 customers
Processed 90000 customers
Processed 100000 customers
Processed 110000 customers
Processed 120000 customers
Processed 130000 customers
Processed 140000 customers
Processed 150000 customers
Processed 160000 customers
Processed 170000 customers
Processed 180000 customers
Processed 190000 customers
Processed 200000 customers
Processed 210000 customers
Processed 220000 customers
Processed 230000 customers
Processed 240000 customers
Processed 250000 customers
Processed 260000 customers
Processed 270000 customers
Processed 280000 customers
Processed 290000 customers
Processed 300000 customers
Processed 310000 customers
Processed 320000 customers
Processed 330000 customers
Processed 340000 customers
Processed 350000 customers
Processed 360000 customers
Processed 37000

In [115]:
reformat_predictions = [[str(v) for v in elem] for elem in predictions]

In [105]:
customers = pd.read_csv('h-and-m-personalized-fashion-recommendations/customers.csv')
customers = customers['customer_id']
customers = customers.to_list()

submission = {'customer_id': [], 'prediction': []}

submission['customer_id'] = customers
submission['prediction'] = [" ".join(elem) for elem in reformat_predictions]

TypeError: sequence item 0: expected str instance, int found

In [116]:
submission['customer_id'] = customers
submission['prediction'] = [" ".join(elem) for elem in reformat_predictions]

In [109]:
second_try = pd.DataFrame(submission)

In [117]:
second_try.to_csv('second_submission.csv', index=False)

In [ ]:
counter = {}

for _, val in res.items():
    for elem, cnt in val:
        if elem not in counter:
            counter[elem] = 0
        counter[elem] += cnt

total_cnt = list(counter.items())
total_cnt = sorted(total_cnt, key=lambda x: x[1], reverse=True)

In [113]:
most_popular_all_time = [elem[0] for elem in total_cnt[:20]]